In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image, PointCloud2
from cv_bridge import CvBridge
import cv2
import numpy as np
import pandas as pd
from time import time, sleep
from datetime import datetime

In [2]:
# Create directories for saving images
import os
left_images_dir = 'left_images'
right_images_dir = 'right_images'
point_clouds_dir = 'point_clouds'
os.makedirs(left_images_dir, exist_ok=True)
os.makedirs(right_images_dir, exist_ok=True)
os.makedirs(point_clouds_dir, exist_ok=True)

In [3]:
class SimpleDataCollector(Node):
    def __init__(self):
        super().__init__('simple_data_collector')
        self.bridge = CvBridge()

        self.last_timestamp = 0
        self.sync_rate = 1/100
        
        self.left_image = None
        self.right_image = None
        self.point_cloud = None

        self.left_time = 0
        self.right_time = 0
        self.point_cloud_time = 0

        self.left_image_sub = self.create_subscription(Image, '/cam_1/zed_node/left/image_rect_color', self.left_image_callback, 10)
        self.right_image_sub = self.create_subscription(Image, '/cam_1/zed_node/right/image_rect_color', self.right_image_callback, 10)
        self.point_cloud_sub = self.create_subscription(PointCloud2, '/cam_1/zed_node/point_cloud/cloud_registered', self.point_cloud_callback, 10)
        
    def synchronization(self):
        #if self.left_image is not None and self.right_image is not None and self.depth_image is not None:
        if self.left_time == self.right_time == self.point_cloud_time:

            cv_left = self.bridge.imgmsg_to_cv2(self.left_image, desired_encoding='bgr8')
            cv_right = self.bridge.imgmsg_to_cv2(self.right_image, desired_encoding='bgr8')
            npy_cloud = self.point_cloud2_to_npy(self.point_cloud)

            left_filename = f"{left_images_dir}/frame_{self.left_time}.png"
            right_filename = f"{right_images_dir}/frame_{self.right_time}.png"
            cloud_filename = f"{point_clouds_dir}/frame_{self.point_cloud_time}.npy"

            cv2.imwrite(left_filename, cv_left)
            cv2.imwrite(right_filename, cv_right)
            np.save(cloud_filename, npy_cloud)
            
            print('Current time is:' + str(self.left_time))
            interval = self.left_time - self.last_timestamp
            print('Time   interval:' + str(interval))

            self.last_timestamp = self.left_time
            
    def left_image_callback(self, msg):
        self.left_image = msg
        self.left_time = self.left_image.header.stamp.sec + self.left_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def right_image_callback(self, msg):
        self.right_image = msg
        self.right_time = self.right_image.header.stamp.sec + self.right_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def point_cloud_callback(self, msg):
        self.point_cloud = msg
        self.point_cloud_time = self.point_cloud.header.stamp.sec + self.point_cloud.header.stamp.nanosec / 1e9
        self.synchronization()
    
    def point_cloud2_to_npy(self, msg):
        from sensor_msgs_py import point_cloud2
        cloud_points = point_cloud2.read_points_numpy(msg, skip_nans=False, field_names=['x','y','z'], reshape_organized_cloud=True)
        return cloud_points

In [4]:
rclpy.init()
simple_data_collector = SimpleDataCollector()
rclpy.spin(simple_data_collector)

Current time is:1707805137.5836985
Time   interval:1707805137.5836985
Current time is:1707805137.6839054
Time   interval:0.10020685195922852
Current time is:1707805137.7172046
Time   interval:0.03329920768737793
Current time is:1707805137.8007195
Time   interval:0.08351492881774902
Current time is:1707805137.8507874
Time   interval:0.050067901611328125
Current time is:1707805137.8840785
Time   interval:0.03329110145568848
Current time is:1707805137.9676554
Time   interval:0.08357691764831543
Current time is:1707805138.0177085
Time   interval:0.05005311965942383
Current time is:1707805138.0844595
Time   interval:0.06675100326538086
Current time is:1707805138.1345835
Time   interval:0.05012392997741699
Current time is:1707805138.1845784
Time   interval:0.04999494552612305
Current time is:1707805138.2347074
Time   interval:0.050128936767578125
Current time is:1707805138.3182786
Time   interval:0.08357119560241699
Current time is:1707805138.4183645
Time   interval:0.10008597373962402
Curre

KeyboardInterrupt: 